In [2]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors

In [3]:
data = pd.read_csv("data.csv",low_memory=False);

In [4]:
descriptors = pd.read_csv("descriptors.csv");

# Import smiles data from txt file

The simplified molecular-input line-entry system (SMILES) is a specification in form of a line notation for describing the structure of chemical species using short ASCII strings. SMILES strings can be imported by most molecule editors for conversion back into two-dimensional drawings or three-dimensional models of the molecules. The smiles data file shows the molecule ids and their respective structure. It was downloaded here:
https://pubchem.ncbi.nlm.nih.gov/bioassay/624202#section=Top

Click on the "Download" tab -> select "tested substances" -> click "Download service" -> Chose "Smiles" as format and download. 

In [5]:
smilesData = pd.read_csv("smilesData.txt",sep="\t", header=None);

In [6]:
smilesData.columns = ['molSID','chemStruc']

In [7]:
descList = [i[0] for i in Descriptors._descList]

In [8]:
data = data.loc[5:,['PUBCHEM_SID','PUBCHEM_ACTIVITY_SCORE']];

In [9]:
inactives= data.loc[data.PUBCHEM_ACTIVITY_SCORE==0]
actives= data.loc[data.PUBCHEM_ACTIVITY_SCORE!=0]
print("Number of inactives is ",len(inactives))
print("Number of actives is ", len(actives))

Number of inactives is  364035
Number of actives is  13515


In [10]:
selectedInactives = inactives.sample(350)
selectedActives = actives.sample(700)

In [11]:
smallDataset = pd.concat([selectedInactives,selectedActives])

In [12]:
smallDataset.PUBCHEM_SID.values

array([ 24827535.,   4247012.,  24785101., ...,    858176.,   4262957.,
        57267033.])

In [13]:
print("Shape of small dataset is",smallDataset.shape)

Shape of small dataset is (1050, 2)


In [14]:
# Convert floating point values to integers. 
smallDataset['PUBCHEM_SID'] = smallDataset['PUBCHEM_SID'].astype(np.int64)
smallDataset['PUBCHEM_ACTIVITY_SCORE'] = smallDataset['PUBCHEM_ACTIVITY_SCORE'].astype(np.int64)

In [15]:
smallDataset.head()

,PUBCHEM_SID,PUBCHEM_ACTIVITY_SCORE
190548,24827535,0
29688,4247012,0
149222,24785101,0
128777,22414026,0
110013,17511624,0


# Method for calculating descriptors

Input the descriptors list and a smiles data tuple where first column contains the molecule substance IDs and the second column contains the chemical structure. E.g.
124899260	"C1=CC=C(C=C1)CCCCOC2=CC=C(C=C2)CC(=O)NO"

The method outputs a matrix where each row contains the molSID and its pertaining descriptors. 

In [16]:
smallSmilesData = smilesData.loc[smilesData.molSID.isin(smallDataset.PUBCHEM_SID.values)]

In [17]:
smallSmilesData.shape

(1050, 2)

In [18]:
for idx,val in enumerate(smallSmilesData.chemStruc.values):
    m = Chem.MolFromSmiles(val)
    descriptor = descList[idx]
    tempDescVal = getattr(Descriptors,descriptor)(m)
    print(tempDescVal)
    break

0


In [19]:
descriptorsMatrix = pd.DataFrame(np.zeros([len(smallSmilesData),len(descList)]))
descriptorsMatrix.columns = descList
descriptorsMatrix.insert(loc=0, column="molSID",value = smallSmilesData.molSID.values)

In [26]:
descriptorsMatrix.loc[descriptorsMatrix.molSID == 124896151,'MaxAbsPartialCharge'] = 2

In [ ]:
for i, mol in enumerate(smallSmilesData.molSID):
    m= Chem.MolFromSmiles(smallSmilesData.chemStruc.values[i])
    for idx, val in enumerate(descList):
        descriptor = descList[idx]
        tempDescVal = getattr(Descriptors,descriptor)(m)
        descriptorsMatrix.loc[descriptorsMatrix.molSID==smallSmilesData.molSID.values[i],descriptor] = tempDescVal 

# Mini dataframe


In [20]:
extraSmallData = smallSmilesData[:5]

In [21]:
exDescM = pd.DataFrame(np.zeros([len(extraSmallData),len(descList)]))
exDescM.columns = descList
exDescM.insert(loc=0, column="molSID",value = extraSmallData.molSID.values)
exDescM.insert(loc=1, column="chemStruc",value = extraSmallData.chemStruc.values)

# mini end

# method for calculating descriptors for full matrix

In [ ]:
for i, mol in enumerate(extraSmallData.molSID):
    m= Chem.MolFromSmiles(extraSmallData.chemStruc.values[i])
    for idx, val in enumerate(descList):
        descriptor = descList[idx]
        tempDescVal = getattr(Descriptors,descriptor)(m)
        exDescM.loc[exDescM.molSID == mol,descriptor] = tempDescVal 
        break
    break

In [ ]:
def calculateDescriptors(descriptorsMatrix):
    descList = list(descriptorsMatrix.columns.values)
    descList.remove('molSID')
    descList.remove('chemStruc')
    chemStruc = exDescM.chemStruc.values
    for i, mol in enumerate(descriptorsMatrix.molSID):
        m= Chem.MolFromSmiles(chemStruc[i])
        for idx, val in enumerate(descList): 
            descriptor = descList[idx]
            tempDescVal = getattr(Descriptors,descriptor)(m)
            descriptorsMatrix.loc[descriptorsMatrix.molSID == mol , descriptor] = tempDescVal 

In [ ]:
calculateDescriptors(exDescM)

In [ ]:
list(exDescM.columns.values)

# Multiprocessing module


In [118]:
dM.head()

,molSID,chemStr,NumRadicalElectrons,FpDensityMorgan3,MinAbsPartialCharge,MinPartialCharge,HeavyAtomMolWt,MaxEStateIndex,MinAbsEStateIndex,qed,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,124897571,COC1=CC(=C(C=C1/C=C/C2=NC=CC3=CC=CC=C32)OC)OC,0.0,0.0,0.163951,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,124897570,COC1=C(C=C(C=C1)/C=C/C2=NC=CC3=CC=CC=C32)OC,0.0,0.0,0.160809,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,124895757,CC(=O)N(C1=CC=C(C=C1)/C=C/2\C=CC3=CC=CC=C32)O,0.0,0.0,0.247057,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,124894820,CN(C)C1=CC(=C(C=C1)/C=C/C2=CC=NC3=CC=CC=C23)OC,0.0,0.0,0.127644,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,124894291,CN(C)C1=CC=C(C=C1)C(=O)OC2=CC(=C(C=C2[N+](=O)[...,0.0,0.0,0.343272,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [112]:
def simpleCalcDesc(molSID):
    chemStr = dM.chemStr.values[dM.molSID==molSID][0]
    m= Chem.MolFromSmiles(chemStr)
    descVal = Descriptors.MinAbsPartialCharge(m)    
    dM.loc[dM.molSID == molSID, 'MinAbsPartialCharge'] = descVal


In [42]:
import time
from multiprocessing import Pool

In [113]:
simpleCalcDesc(124897571)

In [116]:
#from functools import partial


#iterable = list(range(len(npMolSID)))

#func = partial(simpleCalcDesc,)
start = time.time()
pool = mp.Pool()
result = pool.map(simpleCalcDesc,npMolSID)

end = time.time()
pool.close()
print("time to complete: ", end-start)

time to complete:  0.7645409107208252


In [115]:
start = time.time()
for idx,val in enumerate(npChemStr):
    simpleCalcDesc(npMolSID[idx])
end = time.time()
print("time to complete: ", end-start)

time to complete:  2.9460060596466064


In [ ]:
num_partitions = 2 #number of partitions to split dataframe
num_cores = 4 #number of cores on your machine

# Trying things

In [23]:
output = mp.Queue()
mp.cpu_count()

8

In [24]:
dM = pd.DataFrame(np.zeros([len(smallSmilesData),len(descList)]))
dM.columns = descList
dM.insert(loc=0, column="molSID",value = smallSmilesData.molSID.values)
dM.insert(loc=1, column="chemStr",value = smallSmilesData.chemStruc.values)

In [25]:
chemStr = dM.chemStr.values
molSID = dM.molSID.values

In [27]:
def calculateDescriptor(descriptor, chemStr,molSID):
    for idx,val in enumerate(molSID):
        m= Chem.MolFromSmiles(chemStr[idx])
        descVal = Descriptors.MaxAbsPartialCharge(m)
        dM.loc[dM.molSID == val, descriptor] = descVal


In [76]:
dM.loc[dM.molSID == 124897320, 'MinAbsEStateIndex']

0    1.034612
Name: MinAbsEStateIndex, dtype: float64

In [29]:
simpleCalcDesc('MinAbsEStateIndex','CSC1=CC=C(C=C1)/C=C/C2=CC=NC3=CC=CC=C23',124897320)

0.2562218097933055


In [30]:
npMolSID = np.array(molSID)
npChemStr = np.array(chemStr)
npDescript =  ['MolWt']*len(npMolSID);
np.vstack((npMolSID,npChemStr))
paData =np.vstack((npDescript,npChemStr,npMolSID))

In [117]:
dM.head()

,molSID,chemStr,NumRadicalElectrons,FpDensityMorgan3,MinAbsPartialCharge,MinPartialCharge,HeavyAtomMolWt,MaxEStateIndex,MinAbsEStateIndex,qed,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,124897571,COC1=CC(=C(C=C1/C=C/C2=NC=CC3=CC=CC=C32)OC)OC,0.0,0.0,0.163951,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,124897570,COC1=C(C=C(C=C1)/C=C/C2=NC=CC3=CC=CC=C32)OC,0.0,0.0,0.160809,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,124895757,CC(=O)N(C1=CC=C(C=C1)/C=C/2\C=CC3=CC=CC=C32)O,0.0,0.0,0.247057,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,124894820,CN(C)C1=CC(=C(C=C1)/C=C/C2=CC=NC3=CC=CC=C23)OC,0.0,0.0,0.127644,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,124894291,CN(C)C1=CC=C(C=C1)C(=O)OC2=CC(=C(C=C2[N+](=O)[...,0.0,0.0,0.343272,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [88]:
dM.loc[dM.molSID == 124897571, 'NumRadicalElectrons']= 2

In [ ]:
def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [ ]:
result = parallelize_dataframe(exDescM,calculateDescriptors)